In [1]:
## MAKING STRUCTURAL PREDICTIONS WITH CHAI
from pathlib import Path
import numpy as np
import torch
from chai_lab.chai1 import run_inference
import os
gpu_index = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_index
os.environ["OMP_NUM_THREADS"] = "6"

### incorrect guide
example_fasta = f"""
>protein|PIK3CB_AUCGGAUUCAUAUUAGGAGAU_AUCUCCUAAUAUGAAUCCUAU_mismatch_guide_pos3
MYSGAGPALAPPAPPPPIQGYAFKPPPRPDFGTSGRTIKLQANFFEMDIPKIDIYHYELDIKPEKCPRRVNREIVEHMVQHFKTQIFGDRKPVFDGRKNLYTAMPLPIGRDKVELEVTLPGEGKDRIFKVSIKWVSCVSLQALHDALSGRLPSVPFETIQALDVVMRHLPSMRYTPVGRSFFTASEGCSNPLGGGREVWFGFHQSVRPSLWKMMLNIDVSATAFYKAQPVIEFVCEVLDFKSIEEQQKPLTDSQRVKFTKEIKGLKVEITHCGQMKRKYRVCNVTRRPASHQTFPLQQESGQTVECTVAQYFKDRHKLVLRYPHLPCLQVGQEQKHTYLPLEVCNIVAGQRCIKKLTDNQTSTMIRATARSAPDRQEEISKLMRSADFNTDPYVREFGIMVKDEMTDVTGRVLQPPSILYGGRNKAIATPVQGVWDMRNKQFHTGIEIKVWAIACFAPQRQCTEVHLKSFTEQLRKISRDAGMPIQGQPCFCKYAQGADSVEPMFRHLKNTYAGLQLVVVILPGKTPVYAEVKRVGDTVLGMATQCVQMKNVQRTTPQTLSNLCLKINVKLGGVNNILLPQGRPPVFQQPVIFLGADVTHPPAGDGKKPSIAAVVGSMDAHPNRYCATVRVQQHRQEIIQDLAAMVRELLIQFYKSTRFKPTRIIFYRAGVSEGQFQQVLHHELLAIREACIKLEKDYQPGITFIVVQKRHHTRLFCTDKNERVGKSGNIPAGTTVDTKITHPTEFDFYLCSHAGIQGTSRPSHYHVLWDDNRFSSDELQILTYQLCHTYVRCTRSVSIPAPAYYAHLVAFRARYHLVDKEHDAAEGDHTDGQANGRDHQALAKAVQVHQDTLRTMYFA
>rna|PIK3CB_AUCGGAUUCAUAUUAGGAGAU_AUCUCCUAAUAUGAAUCCUAU_mismatch_guide_pos3
AUCGGAUUCAUAUUAGGAGAU
>rna|PIK3CB_AUCGGAUUCAUAUUAGGAGAU_AUCUCCUAAUAUGAAUCCUAU_mismatch_guide_pos3
AUCUCCUAAUAUGAAUCCUAU 
""".strip()

### ACCUCUGUUGUAGGACUAAUCCUAU
### ...
# Write the example FASTA to a file ### REQUIRED
fasta_path = Path("example.fasta")
fasta_path.write_text(example_fasta)


output_dir = Path("outputs")
output_cif_paths = run_inference(
    fasta_file=fasta_path,
    output_dir=output_dir,
    num_trunk_recycles=3,
    num_diffn_timesteps=200,
    seed=42,
    device=torch.device("cuda:0"),  # Use "cpu" if no GPU is available
    use_esm_embeddings=True,
)


pdb_files = list(output_dir.glob("*.pdb"))
print("Generated PDB files:")
for pdb_file in pdb_files:
    print(pdb_file)


/data/home/mrichte3/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/data/home/mrichte3/miniconda3/lib/python3.12/site-packages/torch/export/_unlift.py:58: UserWarning: Attempted to insert a get_attr Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule, GraphModule.add_parameter to add the necessary Parameter, or nn.Module.register_buffer to add the necessary buffer
  getattr_node = gm.graph.get_attr(lifted_node)
/data/home/mrichte3/miniconda3/lib/python3.12/site-packages/torch/fx/graph.py:1460: UserWarning: Node _lifted_tensor_constant0_1 target _lifted_tensor_constant0 _lifted_tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(f'Node {node} target {node.target} {atom} of {seen_qualname} does '
/data/home/mrichte3/miniconda3/lib/python3.12/site-packages/torch/fx/graph.py:1460: UserWarning: Node _lifted_tensor_constant1_1 target _lifted_tensor_constant1 _lifted_tensor_constant1 of  does not re

Writing output to outputs/pred.model_idx_0.cif
Writing output to outputs/pred.model_idx_1.cif
Writing output to outputs/pred.model_idx_2.cif
Writing output to outputs/pred.model_idx_3.cif
Writing output to outputs/pred.model_idx_4.cif
Generated PDB files:
